In [ ]:
!apt-get install p7zip

In [ ]:
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z

In [ ]:
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z

In [ ]:
!unzip ../input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
import os,sys
import pandas as pd
import numpy as np

In [ ]:
root_dir = os.getcwd()

In [ ]:
model_dir = os.path.join(root_dir,'model')

In [ ]:
# read in unpacked training data
df_train = pd.read_csv('train.tsv', sep='\t')

In [ ]:
df_train.shape

In [ ]:
df_train['name']=df_train['name'].astype(str)
df_train['category_name']=df_train['category_name'].astype(str)
df_train['item_description']=df_train['item_description'].astype(str)
df_train['category_text'] = df_train['category_name'].apply(lambda x: ' '.join(str(x).split('/')))
df_train['text'] = df_train[['name','category_text','item_description']].apply(lambda x: ''.join(x), axis=1)
# convert to lowercase
df_train['text'] = df_train['text'] .apply(lambda x: x.lower())

### Load tokenizer

In [ ]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()

In [ ]:
sp.Load('../input/bpe-model-text-lower-10k/bpe_model_text_lower_10k.model')

### Apply tokenizer

In [ ]:
offset = 1
df_train['tokenized'] = df_train['text'].apply(lambda x: [k+offset for k in sp.EncodeAsIds(x)])

In [ ]:
df_train['token_cnt'] = df_train['tokenized'].apply(len)

### Decide about sequence length

In [ ]:
df_train['token_cnt'].quantile(.90)

### Prepare Keras model

In [ ]:
import os,sys
import pandas as pd
import pickle
import numpy as np

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Activation
from keras.layers.embeddings import Embedding
from keras.layers import LSTM

### Convert data into (padded) numpy arrays

In [ ]:
token_truncate_len = 90

In [ ]:
# Create an encoded, truncated array
train_np = pad_sequences(df_train['tokenized'].values,maxlen=token_truncate_len,truncating='post')

In [ ]:
train_np.shape

### Prepare X/Y numpy arrays

In [ ]:
X_train = train_np

In [ ]:
y_train = df_train['price'].values

In [ ]:
max_features = 10000+1

In [ ]:
# arbitrary choice, needs to be refined 
embedding_dimension = 20

In [ ]:
from keras.optimizers import Adam
from keras.losses import mean_squared_logarithmic_error
from keras.layers import Bidirectional

In [ ]:
model=Sequential()
model.add(
    Embedding(
        max_features,
        embedding_dimension,
        input_length=token_truncate_len
    )
)
#model.add(Bidirectional(CuDNNLSTM(embedding_dimension)))
model.add(Bidirectional(LSTM(embedding_dimension)))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('linear'))

opt = Adam(lr=1e-3, decay=1e-3 / 200)

optimizer = opt
model.compile(
    loss = mean_squared_logarithmic_error,
    optimizer=opt)


In [ ]:
%%time
model.fit(
    X_train,
    y_train,
    epochs=1
)

### Export the model

In [ ]:
# export da model
model.save('mercari_bilstm_model.h5')

### Apply model to the train data

In [ ]:
z_train = model.predict(X_train)

In [ ]:
z_train = z_train.reshape(-1,)

### Compute prediction error on the train data

In [ ]:
def rmsle(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.sqrt(np.mean(np.power(np.log1p(y_true + 1) - np.log1p(y_pred + 1), 2)))

In [ ]:
rmsle(y_train,z_train)

### Load the test data 

In [ ]:
df_test = pd.read_csv('test_stg2.tsv', sep='\t')

In [ ]:
df_test['name']=df_test['name'].astype(str)
df_test['category_name']=df_test['category_name'].astype(str)
df_test['item_description']=df_test['item_description'].astype(str)
df_test['category_text'] = df_test['category_name'].apply(lambda x: ' '.join(str(x).split('/')))
df_test['text'] = df_test[['name','category_text','item_description']].apply(lambda x: ''.join(x), axis=1)
# convert to lowercase
df_test['text'] = df_test['text'] .apply(lambda x: x.lower())

In [ ]:
offset = 1
df_test['tokenized'] = df_test['text'].apply(lambda x: [k+offset for k in sp.EncodeAsIds(x)])

In [ ]:
X_test = pad_sequences(df_test['tokenized'].values,maxlen=token_truncate_len,truncating='post')

In [ ]:
X_test.shape

In [ ]:
z_test = model.predict(X_test)
z_test = z_test.reshape(-1,)

### Export test scores for submission

In [ ]:
test_result_df = pd.DataFrame(data={'test_id':df_test.index,'price':z_test})

In [ ]:
# create submission
test_result_df.to_csv("submission.csv", index = False)